<a href="https://colab.research.google.com/github/saishshinde15/Langchain_RAG_BOT/blob/main/Building_Gradio_QnA_Retriver_Langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture --no-stderr
%pip install --quiet -U langchain_core langgraph langchain langchain_community

In [2]:
%pip install -qU langchain_community pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 292.8/292.8 kB 7.3 MB/s eta 0:00:00


In [3]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("/content/HAI_AI-Index-Report-2024.pdf")

In [4]:
docs=loader.load()

In [5]:
%pip install -qU langchain-text-splitters

In [6]:
from langchain_text_splitters import CharacterTextSplitter

In [7]:
text_splitter=CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs=text_splitter.split_documents(docs)

In [8]:
%pip install --upgrade --quiet  langchain sentence_transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 8.5 MB/s eta 0:00:00


In [9]:
import getpass

inference_api_key = getpass.getpass("Enter your HF Inference API Key:\n\n")

Enter your HF Inference API Key:

··········


In [10]:
from langchain_community.embeddings import HuggingFaceInferenceAPIEmbeddings

embeddings = HuggingFaceInferenceAPIEmbeddings(
    api_key=inference_api_key, model_name="sentence-transformers/all-MiniLM-l6-v2"
)

/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_name" in HuggingFaceInferenceAPIEmbeddings has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [11]:
pip install -qU langchain-community faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 37.6 MB/s eta 0:00:00


In [13]:
import faiss
from langchain_community.vectorstores import FAISS

In [14]:
db=FAISS.from_documents(docs,embeddings)

In [15]:
retriever = db.as_retriever()

In [16]:
%pip install -qU langchain-groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 7.3 MB/s eta 0:00:00


In [17]:
import getpass
import os

os.environ["GROQ_API_KEY"] = getpass.getpass()

from langchain_groq import ChatGroq

llm = ChatGroq(model="llama3-8b-8192")

··········


In [18]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)


question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

results = rag_chain.invoke({"input": "What is this report about"})

results

{'input': 'What is this report about',
 'context': [Document(metadata={'source': '/content/HAI_AI-Index-Report-2024.pdf', 'page': 434}, page_content='CHAPTER 9:\nPublic \nOpinionArtificial Intelligence\nIndex Report 2024'),
  Document(metadata={'source': '/content/HAI_AI-Index-Report-2024.pdf', 'page': 426}, page_content='427\nArtificial Intelligence\nIndex Report 2024 Chapter 8 Preview Table of Contents20132016201920220%50%100%\n20132016201920220%50%100%\n20132016201920220%50%100%\n20132016201920220%50%100%\n20132016201920220%50%100%\n20132016201920220%50%100%\n20132016201920220%50%100%\n20132016201920220%50%100%\n20132016201920220%50%100%\n20132016201920220%50%100%\n20132016201920220%50%100%\n20132016201920220%50%100%\n20132016201920220%50%100%\n20132016201920220%50%100%\n20132016201920220%50%100%\n20132016201920220%50%100%\n20132016201920220%50%100%\n20132016201920220%50%100%\n20132016201920220%50%100%\n20132016201920220%50%100%\n20132016201920220%50%100%\nMale\nFemaleNew inf ormati

In [19]:
print(results["context"][0].page_content)

CHAPTER 9:
Public 
OpinionArtificial Intelligence
Index Report 2024


In [20]:


print(results["answer"])


This report, the "Artificial Intelligence Index Report 2024", appears to be about public opinion on artificial intelligence and its education in Europe.


In [29]:
# prompt: similar code for ours:from langchain_community.chat_message_histories import ChatMessageHistory
# from langchain_core.chat_history import BaseChatMessageHistory
# from langchain_core.runnables.history import RunnableWithMessageHistory
# store = {}
# def get_session_history(session_id: str) -> BaseChatMessageHistory:
#     if session_id not in store:
#         store[session_id] = ChatMessageHistory()
#     re

from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

class ChatWithHistory(RunnableWithMessageHistory):
    retriever: object
    llm: object
    prompt: object
    def __init__(self, retriever, llm, prompt):
        super().__init__(get_session_history)
        self.retriever = retriever
        self.llm = llm
        self.prompt = prompt
    def invoke(self, input_str, config=None):
        question_answer_chain = create_stuff_documents_chain(self.llm, self.prompt)
        rag_chain = create_retrieval_chain(self.retriever, question_answer_chain)
        result = rag_chain.invoke({"input": input_str})
        return result


In [30]:
rag_chain.invoke(
    {"input": "What is This report about?"},
    config={
        "configurable": {"session_id": "abc123"}
    },  # constructs a key "abc123" in `store`.
)["answer"]

"This report appears to be about artificial intelligence (AI) and its relationship to public opinion, education, and demographics. Specifically, it discusses the diversity of master's graduates in informatics, computer science, computer engineering, and IT fields in Europe."

In [31]:
rag_chain.invoke(
    {"input": "Did you say artificial intelligence?"},
    config={
        "configurable": {"session_id": "abc123"}
    },  # constructs a key "abc123" in `store`.
)["answer"]

'Yes, I did.'

In [32]:


!pip install -q gradio

import gradio as gr

def qa_function(query):
  results = rag_chain.invoke({"input": query})
  return results["answer"]


iface = gr.Interface(
    fn=qa_function,
    inputs=gr.Textbox(lines=2, placeholder="Enter your question here..."),
    outputs="text",
    title="HAI AI Index Report 2024 Q&A",
    description="Ask questions about the HAI AI Index Report 2024.",
)

iface.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://c5736939dbba964063.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [49]:
import gradio as gr

def qa_function(query):
    results = rag_chain.invoke({"input": query})
    return results["answer"]

# CSS for custom styling
custom_css = """
#title-header {
    font-size: 32px;
    font-weight: bold;
    text-align: center;
    color: #2E8B57;
    margin-bottom: 10px;
}

#description-text {
    font-size: 18px;
    text-align: center;
    margin-bottom: 20px;
    color: #555;
}

.gradio-container {
    background: linear-gradient(135deg, #f3f4f6, #ffffff);
    padding: 50px;
    border-radius: 15px;
    box-shadow: 0px 4px 20px rgba(0, 0, 0, 0.1);
}

.gr-button {
    background-color: #2E8B57;
    color: white;
    font-weight: bold;
    padding: 12px;
    border-radius: 10px;
}

.gr-textbox {
    border: 2px solid #2E8B57;
    border-radius: 8px;
    font-size: 16px;
}
"""

# Create the Gradio interface
iface = gr.Interface(
    fn=qa_function,
    inputs=gr.Textbox(
        lines=2,
        placeholder="Enter your question here...",
        label="Your Question:",
        elem_classes=["gr-textbox"]
    ),
    outputs=gr.Textbox(label="Answer", elem_classes=["gr-textbox"]),
    title="HAI AI Index Report 2024 Q&A",
    description="Ask questions about the HAI AI Index Report 2024.",
    css=custom_css
)

iface.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://59f2a8e348dc12f372.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
